In [1]:
import xml.etree.ElementTree as ET
from glob import glob
import subprocess
import os
import pandas as pd

In [2]:
program = "../../build_release/bin/ssipp"


In [3]:
timeout = 60

In [4]:
target_folder = "../instances/singleagent-icaps2020/den520d/"



In [7]:
results = pd.DataFrame(columns = ["task", "lookahead", "learning algorithm", "solved", "solution length", "solution duration"])
lookaheads = ["10", "20", "100", "200", "400"]
tasks = glob(target_folder+ "/*task.xml")
learnings = ["dijkstralearning", "plrtalearning"]
def parse_output(filepath):
    tree = ET.parse(filepath)
    root = tree.getroot()
    try:
        log = root.find("log")
        agent = log.find("agent")
        path = agent.find("path")
        return float(path.attrib["duration"]), len(path)
    except:
        return float("inf"), 0

In [8]:
for lookahead in lookaheads:
    for task in tasks:
        for learning in learnings:
            print(lookahead, task)
            config = target_folder + "den520d.xml"
            tree = ET.parse(config)
            root = tree.getroot()
            alg = root.find("algorithm")
            learning_algorithm = alg.find("learningalgorithm")
            learning_algorithm.text = learning
            look = alg.find("fixedlookahead")
            look.text = lookahead
            outconfig = config.split("/")[-1]
            tree.write(outconfig)
            obs = task.replace(".xml", "_obs.xml")
            outfile = task.replace(".xml", "_log.xml")
            m = target_folder + "den520d.xml"
            command = [program, task, m, outconfig, obs]
            try:
                subprocess.run(command, timeout = timeout)
            except:
                pass
            res = parse_output(outfile)
            results = results.append(pd.Series(index = results.columns, data = (task, lookahead, learning, True, res[1], res[0])), ignore_index = True)
results["solved"] = results["solution length"] != 0

10 ../instances/singleagent-icaps2020/den520d/40_task.xml
Warning! No 'prioritization' element found inside 'algorithm' section. Its value is set to 'fifo'.
Warning! No 'timelimit' element found inside 'algorithm' section. Its value is set to 'infinity' (no limit).
Warning! No 'rescheduling' element found inside 'algorithm' section. Its value is to 'no'.
Warning! No 'planforturns' element found inside 'algorithm' section. Its value is set to 'false'.
Warning! No 'waitbeforemove' element found inside 'algorithm' section. Its value is set to '0'.
Warning! No 'connectedness' element found inside 'algorithm' section. Its value is set to '5'.
Warning! No 'inflatecollisionintervals' element found inside 'algorithm' section. Its value is set to '0'.
CONFIG LOADED
MAP LOADED
TASK LOADED
OBSTACLES LOADED
Path for agent 0 not found!
1.96602 0 1539890680 21964 0 26 0
LOG STARTED
LOG SAVED
10 ../instances/singleagent-icaps2020/den520d/40_task.xml
Warning! No 'prioritization' element found inside '

CONFIG LOADED
MAP LOADED
TASK LOADED
OBSTACLES LOADED
0.088342 89.6133 0 1 0 0 52
LOG STARTED
LOG SAVED
10 ../instances/singleagent-icaps2020/den520d/94_task.xml
Warning! No 'prioritization' element found inside 'algorithm' section. Its value is set to 'fifo'.
Warning! No 'timelimit' element found inside 'algorithm' section. Its value is set to 'infinity' (no limit).
Warning! No 'rescheduling' element found inside 'algorithm' section. Its value is to 'no'.
Warning! No 'planforturns' element found inside 'algorithm' section. Its value is set to 'false'.
Warning! No 'waitbeforemove' element found inside 'algorithm' section. Its value is set to '0'.
Warning! No 'connectedness' element found inside 'algorithm' section. Its value is set to '5'.
Warning! No 'inflatecollisionintervals' element found inside 'algorithm' section. Its value is set to '0'.
CONFIG LOADED
MAP LOADED
TASK LOADED
OBSTACLES LOADED
0.068722 75.2207 0 1 0 0 34
LOG STARTED
LOG SAVED
10 ../instances/singleagent-icaps2020/d

CONFIG LOADED
MAP LOADED
TASK LOADED
OBSTACLES LOADED
Path for agent 0 not found!
2.23528 0 -1867868104 21966 0 16 0
LOG STARTED
LOG SAVED
10 ../instances/singleagent-icaps2020/den520d/69_task.xml
Warning! No 'prioritization' element found inside 'algorithm' section. Its value is set to 'fifo'.
Warning! No 'timelimit' element found inside 'algorithm' section. Its value is set to 'infinity' (no limit).
Warning! No 'rescheduling' element found inside 'algorithm' section. Its value is to 'no'.
Warning! No 'planforturns' element found inside 'algorithm' section. Its value is set to 'false'.
Warning! No 'waitbeforemove' element found inside 'algorithm' section. Its value is set to '0'.
Warning! No 'connectedness' element found inside 'algorithm' section. Its value is set to '5'.
Warning! No 'inflatecollisionintervals' element found inside 'algorithm' section. Its value is set to '0'.
CONFIG LOADED
MAP LOADED
TASK LOADED
OBSTACLES LOADED
Path for agent 0 not found!
1.97369 0 1560395544 21947

CONFIG LOADED
MAP LOADED
TASK LOADED
OBSTACLES LOADED
Path for agent 0 not found!
1.68363 0 73837656 22071 0 16 0
LOG STARTED
LOG SAVED
10 ../instances/singleagent-icaps2020/den520d/74_task.xml
Warning! No 'prioritization' element found inside 'algorithm' section. Its value is set to 'fifo'.
Warning! No 'timelimit' element found inside 'algorithm' section. Its value is set to 'infinity' (no limit).
Warning! No 'rescheduling' element found inside 'algorithm' section. Its value is to 'no'.
Warning! No 'planforturns' element found inside 'algorithm' section. Its value is set to 'false'.
Warning! No 'waitbeforemove' element found inside 'algorithm' section. Its value is set to '0'.
Warning! No 'connectedness' element found inside 'algorithm' section. Its value is set to '5'.
Warning! No 'inflatecollisionintervals' element found inside 'algorithm' section. Its value is set to '0'.
CONFIG LOADED
MAP LOADED
TASK LOADED
OBSTACLES LOADED
Path for agent 0 not found!
1.79573 0 -1623449064 21919 0

10 ../instances/singleagent-icaps2020/den520d/73_task.xml
Warning! No 'prioritization' element found inside 'algorithm' section. Its value is set to 'fifo'.
Warning! No 'timelimit' element found inside 'algorithm' section. Its value is set to 'infinity' (no limit).
Warning! No 'rescheduling' element found inside 'algorithm' section. Its value is to 'no'.
Warning! No 'planforturns' element found inside 'algorithm' section. Its value is set to 'false'.
Warning! No 'waitbeforemove' element found inside 'algorithm' section. Its value is set to '0'.
Warning! No 'connectedness' element found inside 'algorithm' section. Its value is set to '5'.
Warning! No 'inflatecollisionintervals' element found inside 'algorithm' section. Its value is set to '0'.
CONFIG LOADED
MAP LOADED
TASK LOADED
OBSTACLES LOADED
Path for agent 0 not found!
1.73793 0 2123612152 22040 0 21 0
LOG STARTED
LOG SAVED
10 ../instances/singleagent-icaps2020/den520d/73_task.xml
Warning! No 'prioritization' element found inside '

...

CONFIG LOADED
MAP LOADED
TASK LOADED
OBSTACLES LOADED
4.94733 260.598 0 1 28 35 126
LOG STARTED
LOG SAVED
400 ../instances/singleagent-icaps2020/den520d/5_task.xml
Warning! No 'prioritization' element found inside 'algorithm' section. Its value is set to 'fifo'.
Warning! No 'timelimit' element found inside 'algorithm' section. Its value is set to 'infinity' (no limit).
Warning! No 'rescheduling' element found inside 'algorithm' section. Its value is to 'no'.
Warning! No 'planforturns' element found inside 'algorithm' section. Its value is set to 'false'.
Warning! No 'waitbeforemove' element found inside 'algorithm' section. Its value is set to '0'.
Warning! No 'connectedness' element found inside 'algorithm' section. Its value is set to '5'.
Warning! No 'inflatecollisionintervals' element found inside 'algorithm' section. Its value is set to '0'.
400 ../instances/singleagent-icaps2020/den520d/5_task.xml
Warning! No 'prioritization' element found inside 'algorithm' section. Its value is

CONFIG LOADED
MAP LOADED
TASK LOADED
OBSTACLES LOADED
6.71393 149.63 0 1 309 601 74
LOG STARTED
LOG SAVED
400 ../instances/singleagent-icaps2020/den520d/43_task.xml
Warning! No 'prioritization' element found inside 'algorithm' section. Its value is set to 'fifo'.
Warning! No 'timelimit' element found inside 'algorithm' section. Its value is set to 'infinity' (no limit).
Warning! No 'rescheduling' element found inside 'algorithm' section. Its value is to 'no'.
Warning! No 'planforturns' element found inside 'algorithm' section. Its value is set to 'false'.
Warning! No 'waitbeforemove' element found inside 'algorithm' section. Its value is set to '0'.
Warning! No 'connectedness' element found inside 'algorithm' section. Its value is set to '5'.
Warning! No 'inflatecollisionintervals' element found inside 'algorithm' section. Its value is set to '0'.
CONFIG LOADED
MAP LOADED
TASK LOADED
OBSTACLES LOADED
4.49586 126.958 0 1 318 598 59
LOG STARTED
LOG SAVED
400 ../instances/singleagent-icap

CONFIG LOADED
MAP LOADED
TASK LOADED
OBSTACLES LOADED
0.587299 75.6154 0 1 0 0 32
LOG STARTED
LOG SAVED
400 ../instances/singleagent-icaps2020/den520d/75_task.xml
Warning! No 'prioritization' element found inside 'algorithm' section. Its value is set to 'fifo'.
Warning! No 'timelimit' element found inside 'algorithm' section. Its value is set to 'infinity' (no limit).
Warning! No 'rescheduling' element found inside 'algorithm' section. Its value is to 'no'.
Warning! No 'planforturns' element found inside 'algorithm' section. Its value is set to 'false'.
Warning! No 'waitbeforemove' element found inside 'algorithm' section. Its value is set to '0'.
Warning! No 'connectedness' element found inside 'algorithm' section. Its value is set to '5'.
Warning! No 'inflatecollisionintervals' element found inside 'algorithm' section. Its value is set to '0'.
CONFIG LOADED
MAP LOADED
TASK LOADED
OBSTACLES LOADED
0.636567 75.4754 0 1 0 0 35
LOG STARTED
LOG SAVED
400 ../instances/singleagent-icaps2020

CONFIG LOADED
MAP LOADED
TASK LOADED
OBSTACLES LOADED
2.37384 69.2403 0 1 6 6 33
LOG STARTED
LOG SAVED
400 ../instances/singleagent-icaps2020/den520d/79_task.xml
Warning! No 'prioritization' element found inside 'algorithm' section. Its value is set to 'fifo'.
Warning! No 'timelimit' element found inside 'algorithm' section. Its value is set to 'infinity' (no limit).
Warning! No 'rescheduling' element found inside 'algorithm' section. Its value is to 'no'.
Warning! No 'planforturns' element found inside 'algorithm' section. Its value is set to 'false'.
Warning! No 'waitbeforemove' element found inside 'algorithm' section. Its value is set to '0'.
Warning! No 'connectedness' element found inside 'algorithm' section. Its value is set to '5'.
Warning! No 'inflatecollisionintervals' element found inside 'algorithm' section. Its value is set to '0'.
400 ../instances/singleagent-icaps2020/den520d/79_task.xml
Warning! No 'prioritization' element found inside 'algorithm' section. Its value is 

400 ../instances/singleagent-icaps2020/den520d/45_task.xml
Warning! No 'prioritization' element found inside 'algorithm' section. Its value is set to 'fifo'.
Warning! No 'timelimit' element found inside 'algorithm' section. Its value is set to 'infinity' (no limit).
Warning! No 'rescheduling' element found inside 'algorithm' section. Its value is to 'no'.
Warning! No 'planforturns' element found inside 'algorithm' section. Its value is set to 'false'.
Warning! No 'waitbeforemove' element found inside 'algorithm' section. Its value is set to '0'.
Warning! No 'connectedness' element found inside 'algorithm' section. Its value is set to '5'.
Warning! No 'inflatecollisionintervals' element found inside 'algorithm' section. Its value is set to '0'.
CONFIG LOADED
MAP LOADED
TASK LOADED
OBSTACLES LOADED
4.09259 163.388 0 1 123 227 93
LOG STARTED
LOG SAVED
400 ../instances/singleagent-icaps2020/den520d/45_task.xml
Warning! No 'prioritization' element found inside 'algorithm' section. Its value

In [9]:
results.to_csv("more-results.csv")

In [16]:
for lookahead in lookaheads:
    for learning in learnings:
        print(lookahead, learning)
        print(results.loc[(results.lookahead == lookahead) & (results["learning algorithm"] == learning), "solved"].sum())

10 dijkstralearning
29
10 plrtalearning
30
20 dijkstralearning
39
20 plrtalearning
37
100 dijkstralearning
52
100 plrtalearning
52
200 dijkstralearning
59
200 plrtalearning
56
400 dijkstralearning
62
400 plrtalearning
58


In [11]:
results

,task,lookahead,learning algorithm,solved,solution length,solution duration
0,../instances/singleagent-icaps2020/den520d/40_...,10,dijkstralearning,False,0,inf
1,../instances/singleagent-icaps2020/den520d/40_...,10,plrtalearning,False,0,inf
2,../instances/singleagent-icaps2020/den520d/78_...,10,dijkstralearning,False,0,inf
3,../instances/singleagent-icaps2020/den520d/78_...,10,plrtalearning,False,0,inf
4,../instances/singleagent-icaps2020/den520d/31_...,10,dijkstralearning,False,0,inf
...,...,...,...,...,...,...
995,../instances/singleagent-icaps2020/den520d/16_...,400,plrtalearning,False,0,inf
996,../instances/singleagent-icaps2020/den520d/95_...,400,dijkstralearning,True,2067,2725.05790
997,../instances/singleagent-icaps2020/den520d/95_...,400,plrtalearning,True,141,266.25928
998,../instances/singleagent-icaps2020/den520d/98_...,400,dijkstralearning,True,5304,6468.12740


(inf, 0)

In [105]:
!pwd

/home/devin/Documents/thesis/SituatedSIPP/scripts/20/20
